# Question answering system using SQUAD Dataset.


*   Run this notebook on Python 2.7
*   Use GPU to save time.



Initail setups to be done on Colab.


1.   Runtime --> Change Runtime Type
2.   Runtime Type --> Python 2.7
3. Hardware Accelarator --> GPU



In [1]:
# Run this cell if you want your files to be loaded in your G Drive

from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [33]:
import os

# change this path if you save it different location
path = "/content/drive/My Drive/Cellstrat/Hands-On-Workshops/"

os.chdir(path)
!pwd

/content/drive/My Drive/Cellstrat/Hands-On-Workshops


In [0]:
# Cloning github repositry to drive
! git clone "https://github.com/abdu1azeez/Question-Answering-System-using-SQUAD.git"


os.chdir('./Question-Answering-System-using-SQUAD/code')
print('')
!pwd

Cloning into 'Question-Answering-System-using-SQUAD'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 28 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), done.

/content/drive/My Drive/Cellstrat/Hands On Workshops/Question-Answering-System-using-SQUAD/code



### 1. Requirements

*    Python (Verified on 2.7)
*    tensorflow (Verified on r1.1) or tensorflow-gpu (v1.13.1)
*    numpy
*    nltk
*    tqdm
*    pyprind
*    json

### 2. Preprocessing

Run the following command line in the 'code' folder to preprocess the data:

!./get_started.sh

In this script, the following preprocessing steps will be implemented:

    The SQuAD dataset is downloaded and placed in the data/squad folder. Afterwards, it is tokenized by the tokenizer tool nltk and distributed into four files including contest, question, answer and answer span. Lines in these files are aligned. Each line in the answer span file contains two numbers: the first number refers to the index of the first word of the answer in the context paragraph. The second number is the index of the last word of the answer in the context paragraph.
    
    GloVe word embeddings of dimensionality d = 50, 100, 200, and 300 and vocab size of 400k that have been pretrained on Wikipedia 2014 and Gigaword 5 are downloaded and stored in the data/dwr subfolder. By default, embedding dimensionality of 100 is used.
    
    The vocabulary of the SQuAD dataset is extracted and then the GloVe embedding is accordingly trimmed into a much smaller file.
    
    For SQuAD dataset, the train dataset is splitted into two parts: a 95% slice for training, and the rest 5% for validation purposes. The dev dataset will be used as the test set in this project.


In [0]:
# Giving execute permission to all files
! chmod 766 *

# This command will download all required data files to drive.
!./get_started.sh

     |████████████████████████████████| 345.2MB 59kB/s 
     |████████████████████████████████| 368kB 47.6MB/s 
     |████████████████████████████████| 3.2MB 33.0MB/s 
  Created wheel for pyprind: filename=PyPrind-2.11.2-cp27-none-any.whl size=8371 sha256=12f1b92343655e4fe59c83ce2726330b476b65f12c22fb06772f753e025d8d16
  Stored in directory: /root/.cache/pip/wheels/4c/ed/f4/966a298352e737031124486720b3ec37d7ab4ae57328ee1021
Successfully built pyprind
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.0
    Uninstalling tensorflow-estimator-1.15.0:
      Successfully uninstalled tensorflow-estimator-1.15.0
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uni

In [34]:
os.chdir('./Question-Answering-System-using-SQUAD/code')
print('')
!pwd


/content/drive/My Drive/Cellstrat/Hands-On-Workshops/Question-Answering-System-using-SQUAD/code


In [36]:
!chmod 766 *
!./install_libraries.sh

     |████████████████████████████████| 345.2MB 64kB/s 
     |████████████████████████████████| 368kB 44.8MB/s 
     |████████████████████████████████| 3.2MB 57.0MB/s 
  Created wheel for pyprind: filename=PyPrind-2.11.2-cp27-none-any.whl size=8371 sha256=c45ae5ec53b469b78cae4ae950b77665d5ea90253d76b72866f9ae5b72251f31
  Stored in directory: /root/.cache/pip/wheels/4c/ed/f4/966a298352e737031124486720b3ec37d7ab4ae57328ee1021
Successfully built pyprind
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.0
    Uninstalling tensorflow-estimator-1.15.0:
      Successfully uninstalled tensorflow-estimator-1.15.0
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uni

In [37]:
# Check tensorflow version
!pip show tensorflow-gpu

print('')

# check if GPU is present
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Name: tensorflow-gpu
Version: 1.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python2.7/dist-packages
Requires: tensorflow-estimator, grpcio, mock, keras-applications, enum34, protobuf, keras-preprocessing, gast, tensorboard, wheel, absl-py, backports.weakref, six, numpy, termcolor, astor
Required-by: 

Found GPU at: /device:GPU:0


In [6]:
#
import pandas as pd
import textwrap 

#Change this number to view other documents
Document_Number = 0

temp = pd.read_json('./download/squad/dev-v1.1.json')['data'][Document_Number]

print 'Title: ' + temp['title'] + '\n\n' + 'Paragraph: '

temp = temp['paragraphs'][0]

# Wrap this text. 
wrapper = textwrap.TextWrapper(width=150) 
  
word_list = wrapper.wrap(text=temp['context']) 
  
# Print each line. 
for element in word_list: 
    print(element)
    
print ''

for i in range(5):
  print temp['qas'][i]['question']
  print temp['qas'][i]['answers'][0]['text']
  print 'start_index: %d' %temp['qas'][i]['answers'][0]['answer_start']
  print 'Other possible answers: '
  print temp['qas'][i]['answers'][1:]
  print ''

Title: Super_Bowl_50

Paragraph: 
Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football
Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super
Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th
Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of
naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently
feature the Arabic numerals 50.

Which NFL team represented the AFC at Super Bowl 50?
Denver Broncos
start_index: 177
Other possible answers: 
[{u'text': u'Denver Broncos', u'answer_start': 177}, {u'text': u'Denver Broncos

In [17]:
!python train.py

Number of Epochs to run: 5
Finished reading 81403 training data
Finished reading 4285 val data

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

2019-10-18 13:53:20.478226: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-10-18 13:53:20.530155: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-18 13:53:20.531144: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5602fde5fa20 executing computations on platform CUDA. Devices:
2019-10-18 13:53:20.531184: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla K80, Co

In [26]:
!python qa_answer.py

Number of Epochs to run: 3
File dev-v1.1.json successfully loaded
Preprocessing dev: 100% 48/48 [00:06<00:00,  7.03it/s]

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

2019-10-18 16:39:09.471219: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-10-18 16:39:09.521817: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-18 16:39:09.522711: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55e164bbd9c0 executing computations on platform CUDA. Devices:
2019-10-18 16:39:09.522757: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor

In [27]:
!python evaluate.py data/squad/dev-v1.1.json dev-prediction.json

{"f1": 43.915418313611966, "exact_match": 30.085146641438033}


In [0]:
def generate_custom_question(context, question):
  if len(question) >= 2:  
    json_object = {'version': 1.1,
         'data': [{ 'title': '',
                   'paragraphs': [{'context': context,
                                   'qas':list({'id': i, 'question': question[i]} for i in range(len(question)))}]}]}
    return json_object

  else:
    raise Exception('Ask atleast two questions')

import json

with open('sample.json', 'w') as json_file:
  json.dump(generate_custom_question(temp['context'], ['Which NFL team represented the AFC at Super Bowl 50?',
                                                      'Which NFL team represented the NFC at Super Bowl 50?']), json_file)
  

In [74]:
pd.read_json('sample.json')

,data,version
0,"{u'title': u'', u'paragraphs': [{u'context': u...",1.1


In [70]:
!python answer.py

Number of Epochs to run: 1
File sample.json successfully loaded
Preprocessing dev: 100% 1/1 [00:00<00:00, 15.15it/s]

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

2019-10-19 03:35:01.257375: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-10-19 03:35:01.310101: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-19 03:35:01.311035: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55fd71c33de0 executing computations on platform CUDA. Devices:
2019-10-19 03:35:01.311068: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor dev